In [1]:
# Update metadata to include behavioral task and brain region for recording
# Reference:  conver_all.py

import pynwb
from nwbinspector import inspect_nwbfile, Importance
from nwbinspector.inspector_tools import format_messages

import matlab.engine
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil import tz
from pathlib import Path
import uuid
import math

from oconnor_lab_to_nwb.scripts.load_mat_struct import loadmat
from oconnor_lab_to_nwb.scripts.misc_data import (
    tg_units, 
    crossmodal_units,
    seqlick_units
)
from oconnor_lab_to_nwb.scripts.utils import (
    get_all_mat_files,
    make_trials_times,
    convert_behavior_continuous_variables, 
    convert_ecephys, 
    convert_spike_times, 
    convert_trials,
    get_trials_recordings_time_offsets
)

eng = matlab.engine.start_matlab()
msessionexplorer_path = '/Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_struct/'

dataset_name = "crossmodal"  # tg, crossmodal, seqlick
units_map = crossmodal_units
behavior_task = "cross-modal sensory selection task"
dir_path = '/Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_struct/'
output_dir = '/Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_NWB/'
experimenters = ["Yi-Ting Chang"]

In [2]:
# Get all .mat files for conversion
all_files = list()
for pp in Path(dir_path).glob("*"):
    if pp.name.endswith(".mat"):
        all_files.append(pp.name)

for fi, file_name in enumerate(all_files):

    data_path = dir_path + file_name
    data = loadmat(data_path)["struct_version"]
    base_time_offset = 0

    # First checks of content
    for n in ['tableName', 'tableType', 'tableData', 'referenceTime', 'epochInd', 'userData']:
        if n not in data.keys():
            raise Exception(f"{n} nor found in data: {data_path}")
    print(f"All checks OK for data: {data_path}")

    # Get metadata
    recording_id = file_name.split(".mat")[0]
    session_start_date_str = data["userData"]["sessionInfo"]["seshDate"]  #yymmdd
    session_start_time_str = data["userData"]["sessionInfo"]["session_start_time"]  #hhmmss
    session_start_time = datetime.strptime(session_start_date_str + " " + session_start_time_str, "%y%m%d %H%M%S").replace(tzinfo=tz.gettz("US/Eastern"))
    recSite = data["userData"]["sessionInfo"]["recSite"]
    exp_info = 'electrophysiological recording of ' + recSite + ' during the ' + behavior_task

    init_metadata = dict(
        session_description=data["userData"]["sessionInfo"]["MouseName"] + "_" + data["userData"]["sessionInfo"]["seshDate"],
        identifier=str(uuid.uuid4()),
        session_start_time=session_start_time,
        experiment_description = exp_info,
        lab="O'Connor lab",
        institution="Johns Hopkins University",
        experimenter=experimenters,
    )

    subject_metadata = dict(
        subject_id=data["userData"]["sessionInfo"]["MouseName"],
        date_of_birth=datetime.strptime(data["userData"]["sessionInfo"]["DoB"], "%y%m%d").replace(tzinfo=tz.gettz("US/Eastern")),
        description="no description",
        species="Mus musculus",
        genotype=data["userData"]["sessionInfo"]["Genotype"],
        sex="M" if data["userData"]["sessionInfo"]["sex"] == "male" else "F"
    )


    # Create nwbfile with initial metadata
    nwbfile = nwbfile = pynwb.NWBFile(**init_metadata)

    # Add subject
    nwbfile.subject = pynwb.file.Subject(**subject_metadata)

    # Convert trials data
    trials_recordings_time_offsets = get_trials_recordings_time_offsets(
        data=data, 
        dataset_name=dataset_name,
        base_time_offset=base_time_offset
    )
    trials_times = make_trials_times(
        data=data, 
        trials_recordings_time_offsets=trials_recordings_time_offsets,
        dataset_name=dataset_name
    )
    trials_data = data["tableData"][np.where(data["tableType"] == "eventValues")[0][0]]
    
    events_data = None
    behavior_events_ind = np.where(data["tableName"] == "behavTime")[0]
    if len(behavior_events_ind) > 0:
        events_data = data["tableData"][behavior_events_ind[0]]
    
    convert_trials(
        trials_data=trials_data, 
        events_data=events_data,
        trials_times=trials_times,
        trials_recordings_time_offsets=trials_recordings_time_offsets,
        nwbfile=nwbfile
    )

    # Convert timeseries data
    for n, t, d in zip(data["tableName"], data["tableType"], data["tableData"]): 
        if t == "timeSeries":
            if n == "LFP": 
                if dataset_name == "crossmodal":
                    convert_ecephys(
                        ts_data=d, 
                        trials_times=trials_times,
                        nwbfile=nwbfile, 
                        extra_data=data["userData"],
                        dataset_name=dataset_name,
                        time_column="time"
                    )
            else:
                convert_behavior_continuous_variables(
                    ts_data=d, 
                    trials_times=trials_times,
                    nwbfile=nwbfile, 
                    units_map=units_map,
                    time_column="time"
                )
    
    # Convert spiking data
    ind_t = np.where(data["tableType"] == "eventTimes")[0]
    ind_n = np.where([tn in ["spikeTime", "spikeTimes"] for tn in data["tableName"]])[0]
    ind_spks = np.intersect1d(ind_t, ind_n)
    if len(ind_spks) > 0:
        spiking_data = data["tableData"][ind_spks[0]]
        convert_spike_times(
            spiking_data=spiking_data, 
            trials_times=trials_times,
            trials_recordings_time_offsets=trials_recordings_time_offsets,
            dataset_name=dataset_name,
            nwbfile=nwbfile
        )           

    # Save nwb file
    output_file = output_dir + f"{dataset_name}_{recording_id}.nwb"
    with pynwb.NWBHDF5IO(output_file, "w") as io:
        io.write(nwbfile)

    print(f"Data successfully converted: {output_file}")

    # Inspect converted data
    inspection_generator = inspect_nwbfile(
        nwbfile_path=output_file, 
        importance_threshold=Importance.BEST_PRACTICE_VIOLATION
    )
    inspection_messages = list(inspection_generator)
    if len(inspection_messages) == 0:
        print(f"No violations found for: {output_file}")
        print()
    else:
        print("\n".join(format_messages(inspection_messages, levels=["importance", "file_path"])))
        print("##################################################################################")
        print()

All checks OK for data: /Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_struct/YT071_190722.mat
Data successfully converted: /Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_NWB/crossmodal_YT071_190722.nwb
No violations found for: /Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_NWB/crossmodal_YT071_190722.nwb

All checks OK for data: /Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_struct/EF0148_190228.mat
Data successfully converted: /Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_NWB/crossmodal_EF0148_190228.nwb
No violations found for: /Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_NWB/crossmodal_EF0148_190228.nwb

All checks OK for data: /Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_struct/YT071_190721.mat
Data successfully converted: /Users/yiting/Documents/JHU_NeuroPhD/OConnorLab/Chang_et_al/NWB/CM_ephy_NWB/crossm